In [21]:
import datetime
import pandas as pd 
import json
import requests

def xml_to_dict(root_or_str, strict=True):
    """
    Converts `root_or_str` which can be parsed xml or a xml string to dict.
    """
    root = root_or_str
    if isinstance(root, str):
        import xml.etree.cElementTree as ElementTree
        root = ElementTree.XML(root_or_str)
    return {root.tag: _from_xml(root, strict)}

def dict_to_xml(dict_xml):
    """
    Converts `dict_xml` which is a python dict to corresponding xml.
    """
    return _to_xml(dict_xml)

# Functions below this line are implementation details.
# Unless you are changing code, don't bother reading.
# The functions above constitute the user interface.

def _to_xml(el):
    """
    Converts `el` to its xml representation.
    """
    val = None
    if isinstance(el, dict):
        val = _dict_to_xml(el)
    elif isinstance(el, bool):
        val = str(el).lower()
    else:
        val = el
    if val is None: val = 'null'
    return val

def _extract_attrs(els):
    """
    Extracts attributes from dictionary `els`. Attributes are keys which start
    with '@'
    """
    if not isinstance(els, dict):
        return ''
    return ''.join(' %s="%s"' % (key[1:], value) for key, value in els.iteritems()
                   if key.startswith('@'))

def _dict_to_xml(els):
    """
    Converts `els` which is a python dict to corresponding xml.
    """
    def process_content(tag, content):
        attrs = _extract_attrs(content)
        text = isinstance(content, dict) and content.get('#text', '') or ''
        return '<%s%s>%s%s</%s>' % (tag, attrs, _to_xml(content), text, tag)

    tags = []
    for tag, content in els.iteritems():
        # Text and attributes
        if tag.startswith('@') or tag == '#text' or tag == '#value':
            continue
        elif isinstance(content, list):
            for el in content:
                tags.append(process_content(tag, el))
        elif isinstance(content, dict):
            tags.append(process_content(tag, content))
        else:
            tags.append('<%s>%s</%s>' % (tag, _to_xml(content), tag))
    return ''.join(tags)

def _str_to_datetime(date_str):
    try:
        val = datetime.datetime.strptime(date_str,  "%Y-%m-%dT%H:%M:%SZ")
    except ValueError:
        val = date_str
    return val

def _str_to_boolean(bool_str):
    if bool_str.lower() != 'false' and bool(bool_str):
        return True
    return False

def _from_xml(el, strict):
    """
    Extracts value of xml element element `el`.
    """
    val = None
    # Parent node.
    if len(el):
        val = {}
        for e in el:
            tag = e.tag
            v = _from_xml(e, strict)
            if tag in val:
                # Multiple elements share this tag, make them a list
                if not isinstance(val[tag], list):
                    val[tag] = [val[tag]]
                val[tag].append(v)
            else:
                # First element with this tag
                val[tag] = v
    # Simple node.
    else:
        attribs = el.items()
        # An element with attributes.
        if attribs and strict:
            val = dict(('@%s' % k, v) for k, v in dict(attribs).iteritems())
            if el.text:
                converted = _val_and_maybe_convert(el)
                val['#text'] = el.text
                if converted != el.text:
                    val['#value'] = converted
        elif el.text:
            # An element with no subelements but text.
            val = _val_and_maybe_convert(el)
        elif attribs:
            val = dict(attribs)
    return val

def _val_and_maybe_convert(el):
    """
    Converts `el.text` if `el` has attribute `type` with valid value.
    """
    text = el.text.strip()
    data_type = el.get('type')
    convertor = _val_and_maybe_convert.convertors.get(data_type)
    if convertor:
        return convertor(text)
    else:
        return text
_val_and_maybe_convert.convertors = {
    'boolean': _str_to_boolean,
    'datetime': _str_to_datetime,
    'integer': int
}

# se genera el xml 'a mano con los datos de 'Algete'
myxml='''<escrutinio_sitio>
<porciento_escrutado>100</porciento_escrutado>
<nombre_sitio>Algete</nombre_sitio>
<convocatoria>2019</convocatoria>
<ts>0</ts>
<tipo_sitio>5</tipo_sitio>
<votos>
<contabilizados>
<cantidad>9817</cantidad>
<porcentaje>67.63</porcentaje>
</contabilizados>
<abstenciones>
<cantidad>4698</cantidad>
<porcentaje>32.37</porcentaje>
</abstenciones>
<nulos>
<cantidad>34</cantidad>
<porcentaje>0.35</porcentaje>
</nulos>
<blancos>
<cantidad>48</cantidad>
<porcentaje>0.49</porcentaje>
</blancos>
</votos>
<resultados>
<numero_partidos>15</numero_partidos>
<partido>
<id_partido>86</id_partido>
<nombre>PSOE</nombre>
<votos_numero>2415</votos_numero>
<votos_porciento>24.69</votos_porciento>
</partido>
<partido>
<id_partido>18</id_partido>
<nombre>Cs</nombre>
<votos_numero>2398</votos_numero>
<votos_porciento>24.51</votos_porciento>
</partido>
<partido>
<id_partido>78</id_partido>
<nombre>PP</nombre>
<votos_numero>2073</votos_numero>
<votos_porciento>21.19</votos_porciento>
</partido>
<partido>
<id_partido>104</id_partido>
<nombre>VOX</nombre>
<votos_numero>1184</votos_numero>
<votos_porciento>12.1</votos_porciento>
</partido>
<partido>
<id_partido>41</id_partido>
<nombre>MÁS MADRID</nombre>
<votos_numero>1043</votos_numero>
<votos_porciento>10.66</votos_porciento>
</partido>
<partido>
<id_partido>74</id_partido>
<nombre>PODEMOS-IU</nombre>
<votos_numero>441</votos_numero>
<votos_porciento>4.51</votos_porciento>
</partido>
<partido>
<id_partido>54</id_partido>
<nombre>PACMA</nombre>
<votos_numero>120</votos_numero>
<votos_porciento>1.23</votos_porciento>
</partido>
<partido>
<id_partido>102</id_partido>
<nombre>UPYD</nombre>
<votos_numero>16</votos_numero>
<votos_porciento>0.16</votos_porciento>
</partido>
<partido>
<id_partido>27</id_partido>
<nombre>FE de las JONS</nombre>
<votos_numero>11</votos_numero>
<votos_porciento>0.11</votos_porciento>
</partido>
<partido>
<id_partido>62</id_partido>
<nombre>PCTE</nombre>
<votos_numero>10</votos_numero>
<votos_porciento>0.1</votos_porciento>
</partido>
<partido>
<id_partido>64</id_partido>
<nombre>P-LIB</nombre>
<votos_numero>8</votos_numero>
<votos_porciento>0.08</votos_porciento>
</partido>
<partido>
<id_partido>87</id_partido>
<nombre>PUM+J</nombre>
<votos_numero>7</votos_numero>
<votos_porciento>0.07</votos_porciento>
</partido>
<partido>
<id_partido>63</id_partido>
<nombre>PH</nombre>
<votos_numero>6</votos_numero>
<votos_porciento>0.06</votos_porciento>
</partido>
<partido>
<id_partido>59</id_partido>
<nombre>PCAS-TC</nombre>
<votos_numero>2</votos_numero>
<votos_porciento>0.02</votos_porciento>
</partido>
<partido>
<id_partido>100</id_partido>
<nombre>ULEG</nombre>
<votos_numero>1</votos_numero>
<votos_porciento>0.01</votos_porciento>
</partido>
</resultados>
</escrutinio_sitio>'''
x=xml_to_dict(myxml)
'''
df = pd.DataFrame(x)
df_resultados = pd.DataFrame(df['escrutinio_sitio']['resultados'])
df_resultados
'''

df_votos = pd.DataFrame(x['escrutinio_sitio']['resultados']['partido'])
#df_votos['ciudad'] = x['escrutinio_sitio']['nombre_sitio']
df_votos.drop(columns= 'id_partido', inplace=True)
df_votos = df_votos[df_votos['nombre'].isin(['PSOE','PP','VOX','MÁS MADRID','PODEMOS-IU'])]
df_votos= df_votos.transpose()
df_votos

,0,2,3,4,5
nombre,PSOE,PP,VOX,MÁS MADRID,PODEMOS-IU
votos_numero,2415,2073,1184,1043,441
votos_porciento,24.69,21.19,12.1,10.66,4.51


In [22]:
url = "https://rsl00.epimg.net/elecciones/2019/autonomicas/12/28/09.xml2"
string = requests.get(url)
string

<Response [200]>

In [ ]:
parsear -web scraping 
reponse (200) -> ack de la petición 
librería beautifulsoup -> tratar texto web
.prettify() -> interpreta los caracteres de separación para poner bonito el texto
